In [70]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
import matplotlib.pyplot as plt 
from sklearn.naive_bayes import MultinomialNB
%matplotlib notebook 
import os
import utils
reload(utils)

<module 'utils' from 'utils.pyc'>

In [2]:
X,supply,demand=utils.LoadDataset('./training_set/DesignMatrices/')
X=X.drop(['Temperature','PM2.5'],axis=1) #Dropping other two features for now 
Xnp,supplynp,demandnp=X.values,supply.values,demand.values

In [3]:
Xtrain, supplyTrain, demandTrain, Xval, supplyVal, demandVal=utils.SplitByRegion(Xnp,supplynp,demandnp,.7)


In [4]:
print Xtrain.shape, supplyTrain.shape, demandTrain.shape 
print Xval.shape, supplyVal.shape,demandVal.shape 

(99981L, 3L) (99981L,) (99981L,)
(42899L, 3L) (42899L,) (42899L,)


In [5]:
def one_hot_encode(x, n_classes):
    """
    One hot encode a list of sample labels. Return a one-hot encoded vector for each label.
    : x: List of sample Labels
    : return: Numpy array of one-hot encoded labels
     """
    return np.eye(n_classes)[x]

In [7]:
regID_ohe = one_hot_encode(Xtrain[:,0],n_classes=67)[:,1:]

In [8]:
weatherID_ohe = one_hot_encode(Xtrain[:,2],10)[:,1:]
weatherID_ohe

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [10]:
def myTransform(X, d):
    """One-hot-encodes region number and weather, while raises timeslot to degree d"""
    a = one_hot_encode(X[:,0],n_classes=67)[:,1:]
    c = one_hot_encode(X[:,2],n_classes=10)[:,1:]
    b = X[:,1][:, np.newaxis]
    b = PolynomialFeatures(degree=d).fit_transform(b)
    
    out=np.concatenate((a, b, c), axis=1)
    return out 

In [141]:
bestscore = np.inf 
bestmodel = None
bestdeg = None
for d in np.arange(1,10+1):
    Xd = myTransform(Xtrain, d)
    model=linear_model.RidgeCV(alphas=np.logspace(1,20,5),)
    model.fit(Xd, demandTrain-supplyTrain)
    
    Xd_ = myTransform(Xval, d)
    ypreds = model.predict(Xd_)
    ypreds2 = model.predict(Xd)
    
    score = utils.MeanAbsoluteError(ypreds,demandVal-supplyVal)
    score2 = utils.MeanAbsoluteError(ypreds2,demandTrain-supplyTrain)
    if bestscore > score: 
        bestscore = score 
        bestmodel = model 
        bestdeg = d
    print 'degree = {}, score = {}, score2 = {}'.format(d, score, score2)

degree = 1, score = 10.605072724, score2 = 10.7924705706
degree = 2, score = 10.9580662803, score2 = 11.122287335
degree = 3, score = 612.673367568, score2 = 613.228735715
degree = 4, score = 5525826.4033, score2 = 5522144.73516
degree = 5, score = 1.39577890617e+11, score2 = 1.3932134284e+11
degree = 6, score = 1.76694569678e+15, score2 = 1.76195449109e+15
degree = 7, score = 3.83689822602e+19, score2 = 3.82394317559e+19
degree = 8, score = 6.71889299048e+23, score2 = 6.69351178741e+23
degree = 9, score = 1.05548745927e+23, score2 = 1.05142334999e+23
degree = 10, score = 4.20273330494e+22, score2 = 4.18711874875e+22


10.0

In [142]:
Xfull = np.concatenate((Xtrain, Xval))
Xfull.shape 

(142880L, 3L)

In [144]:
supplyFull = np.concatenate((supplyTrain, supplyVal))
demandFull = np.concatenate((demandTrain, demandVal))

In [147]:
Xfullnew = myTransform(Xfull, bestdeg)

In [149]:
fullmodel = linear_model.RidgeCV()
fullmodel.fit(Xfullnew, demandFull-supplyFull)

RidgeCV(alphas=(0.1, 1.0, 10.0), cv=None, fit_intercept=True, gcv_mode=None,
    normalize=False, scoring=None, store_cv_values=False)

In [150]:
wtrain = utils.ConstructWeatherMatrix('./training_set/weather_data/')
wtrain = wtrain.drop('2016-01-21.csv', axis=1).dropna()

In [151]:
wtrain 

,2016-01-01.csv,2016-01-02.csv,2016-01-03.csv,2016-01-04.csv,2016-01-05.csv,2016-01-06.csv,2016-01-07.csv,2016-01-08.csv,2016-01-09.csv,2016-01-10.csv,2016-01-11.csv,2016-01-12.csv,2016-01-13.csv,2016-01-14.csv,2016-01-15.csv,2016-01-16.csv,2016-01-17.csv,2016-01-18.csv,2016-01-19.csv,2016-01-20.csv
Timeslot,,,,,,,,,,,,,,,,,,,,
2,1.0,2.0,4.0,2.0,6.0,4.0,4.0,4.0,2.0,3.0,4.0,3.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,2.0
3,1.0,2.0,4.0,2.0,4.0,4.0,4.0,4.0,2.0,3.0,4.0,3.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,2.0
4,1.0,2.0,4.0,2.0,4.0,4.0,4.0,2.0,2.0,3.0,4.0,3.0,3.0,2.0,3.0,2.0,4.0,3.0,2.0,2.0
5,1.0,2.0,4.0,2.0,6.0,6.0,4.0,2.0,2.0,2.0,4.0,2.0,3.0,2.0,3.0,2.0,4.0,3.0,2.0,2.0
6,1.0,2.0,4.0,2.0,8.0,8.0,4.0,2.0,2.0,2.0,4.0,2.0,3.0,2.0,3.0,2.0,4.0,3.0,1.0,2.0
7,1.0,2.0,4.0,2.0,8.0,8.0,4.0,2.0,2.0,2.0,4.0,2.0,3.0,2.0,3.0,2.0,4.0,3.0,1.0,2.0
8,1.0,2.0,4.0,2.0,8.0,8.0,4.0,2.0,2.0,2.0,4.0,2.0,3.0,2.0,3.0,2.0,4.0,3.0,1.0,2.0
9,1.0,2.0,4.0,2.0,8.0,8.0,4.0,2.0,2.0,2.0,4.0,2.0,3.0,2.0,3.0,2.0,4.0,3.0,1.0,2.0
10,1.0,2.0,4.0,2.0,8.0,5.0,4.0,2.0,2.0,2.0,4.0,2.0,3.0,2.0,3.0,2.0,4.0,3.0,1.0,2.0


In [153]:
fnames = [x for x in sorted(os.listdir('./test_set/DesignMatrices/')) if x[0]!='.']

for fname in fnames: 
    Xtestpd = pd.read_csv('./test_set/DesignMatrices/'+fname)
    Xtestpd = Xtestpd[Xtestpd.Timeslot <= 51]
    
    Xtest = Xtestpd[['RegionID','Timeslot', 'Weather']].values
    #print np.unique(Xtest[:,1])
    
    if Xtest.size==0:
        continue
    daystopredict= np.unique(Xtest[np.isnan(Xtest[:,2])][:,1])
    for daytopredict in daystopredict:
        wtest=pd.read_csv('./test_set/weather_data/wmatrix_'+fname, header=None, names=['Timeslot','Weather'],
                         index_col='Timeslot')
        wpred = 4 #PredictWeather(wtrain, wtest, daytopredict)
        Xtest[np.isnan(Xtest[:,2]) & (Xtest[:,1]==daytopredict)] = wpred
        print wpred 
    Xtest = myTransform(Xtest.astype(int), bestdeg)
    
    ypreds = fullmodel.predict(Xtest)
    
    utils.WriteOnKaggleFormat(Xtestpd, ypreds, fname.split('.')[0])

4
4


In [36]:
Xtestpd.columns

Index([u'Unnamed: 0', u'Timeslot', u'RegionHash', u'Date', u'RegionID',
       u'Weather', u'Temperature', u'PM2.5'],
      dtype='object')

In [33]:
def PredictWeather(wtrain, wtest, day):
    
    pass

In [54]:
daystopredict 

array([ 44.,  45.])

In [55]:
wtest 

,Weather
Timeslot,
43,4
68,3
80,3
91,3
92,3
104,3
116,2
128,1
139,2


In [89]:
daytopredict = daystopredict[0].astype(int)
daytopredict

44

In [90]:
labels = wtrain.ix[daytopredict].values.astype(int)
labels 

array([1, 2, 2, 8, 4, 4, 4, 2, 8, 8, 4, 4, 2, 2, 2, 2, 8, 2, 2, 1])

In [91]:
orgidx = wtest.index 
orgidx 
wtrainnew = wtrain.ix[orgidx].dropna()
newidx = wtrainnew.index 

In [92]:
newidx 

Int64Index([43, 128, 139, 140, 141], dtype='int64', name=u'Timeslot')

In [95]:
NB_Xtrain = wtrain.ix[newidx].values.T

In [119]:
NB_Ytrain = wtrain.ix[daytopredict].values.astype(int)

In [120]:
classifier = MultinomialNB()

In [121]:
classifier.fit(NB_Xtrain, NB_Ytrain)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [122]:
NB_Xtrain.shape, NB_Ytrain.shape 

((20L, 5L), (20L,))

In [123]:
NB_Xtest=wtest.ix[newidx].values.T

In [124]:
classifier.predict(NB_Xtest)

array([8])

In [125]:
def PredictWeather(wtrain, wtest, daytopredict):
    orgidx = wtest.index 
    wtrainnew = wtrain.ix[orgidx].dropna()
    newidx = wtrainnew.index 
    
    NB_Xtrain = wtrain.ix[newidx].values.T
    NB_Ytrain = wtrain.ix[daytopredict].values.astype(int)
    NB_Xtest=wtest.ix[newidx].values.T
    
    classifier = MultinomialNB()
    classifier.fit(NB_Xtrain, NB_Ytrain)
    
    return classifier.predict(NB_Xtest)

In [126]:
PredictWeather(wtrain, wtest, 4)

array([4])

In [128]:
wtest.ix[newidx]

,Weather
Timeslot,
43,4
128,1
139,2
140,2
141,2


In [129]:
NB_Xtrain

array([[ 1.,  2.,  2.,  2.,  2.],
       [ 2.,  2.,  4.,  4.,  4.],
       [ 2.,  2.,  2.,  2.,  2.],
       [ 8.,  3.,  4.,  4.,  4.],
       [ 4.,  4.,  4.,  4.,  4.],
       [ 4.,  3.,  4.,  4.,  4.],
       [ 4.,  2.,  4.,  4.,  4.],
       [ 2.,  2.,  2.,  2.,  2.],
       [ 5.,  3.,  3.,  3.,  3.],
       [ 8.,  4.,  4.,  2.,  4.],
       [ 4.,  2.,  3.,  2.,  3.],
       [ 4.,  2.,  3.,  2.,  3.],
       [ 2.,  2.,  2.,  2.,  2.],
       [ 2.,  2.,  3.,  3.,  3.],
       [ 2.,  2.,  2.,  2.,  2.],
       [ 2.,  3.,  4.,  4.,  4.],
       [ 8.,  2.,  3.,  3.,  3.],
       [ 2.,  2.,  2.,  2.,  2.],
       [ 2.,  2.,  2.,  2.,  2.],
       [ 1.,  2.,  2.,  2.,  2.]])